In [ ]:
import config
import irc.bot
import requests
import sys, re
import commands
import cmdlist
cmdlist = cmdlist.cmds


class TwitchBot(irc.bot.SingleServerIRCBot):


        def __init__(self, username, client_id, token, channel):


                self.client_id = client_id  
                self.token = token  

                self.channel = '#' + channel

                url = 'https://api.twitch.tv/kraken/users?login=' + channel  
                headers = {'Client-ID': client_id, 'Accept': 'application/vnd.twitchtv.v5+json'}  
                r = requests.get(url, headers=headers).json()  
                self.channel_id = r['users'][0]['_id']  
                print(r)
                print(self.channel_id)
                server = 'irc.chat.twitch.tv'
                port = 6667
                print('서버 ' + server + ', 포트 ' + str(port) + '에 연결중...')
                irc.bot.SingleServerIRCBot.__init__(self, [(server, port, 'oauth:' + token)], username, username)  


        def on_welcome(self, c, e):
            
            
            print(self.channel + '에 연결되었습니다.')  

            c.cap('REQ', ':twitch.tv/membership')  
            c.cap('REQ', ':twitch.tv/tags')  
            c.cap('REQ', ':twitch.tv/commands')  
            c.join(self.channel)  

        def on_pubmsg(self, c, e):  
            # If a chat message starts with an exclamation point, try to run it as a command
            if e.arguments[0][:1] == '!':
                cmd = e.arguments[0][1:]
                self.do_command(e, cmd)
            return

        def do_command(self, e, cmd):
            
            c = self.connection

            cmd = cmd.split(' ')
            if cmd[0] in cmdlist["kuuwang"]:
                c.privmsg(self.channel, commands.msgs.kuuwang())
            if cmd[0] in cmdlist["survival"]:
                c.privmsg(self.channel, commands.msgs.survival())
            if cmd[0] in cmdlist["cmdlists"]:
                c.privmsg(self.channel, commands.msgs.cl())
            if cmd[0] in cmdlist["discord"]:
                c.privmsg(self.channel, commands.msgs.discord())
            if cmd[0] in cmdlist["nintendo"]:
                c.privmsg(self.channel, commands.msgs.ns())
            if cmd[0] in cmdlist["title"]:
                c.privmsg(self.channel, commands.twitchAPI.title())
            if cmd[0] in cmdlist["game"]:
                c.privmsg(self.channel, commands.twitchAPI.game())
            if cmd[0] in cmdlist["uptime"]:
                c.privmsg(self.channel, commands.twitchAPI.uptime())
            if cmd[0] in cmdlist["choice"]:
                commands.rand.choice(self, e, cmd)
            if cmd[0] in cmdlist["dice"]:
                commands.rand.dice(self, e, cmd)
            if cmd[0] in cmdlist["time"]:
                c.privmsg(self.channel, commands.tz.do())
            if cmd[0] in cmdlist["followage"]:
                commands.followage.do(self, e, cmd)
            if cmd[0] in cmdlist["cal"]:
                c.privmsg(self.channel, commands.cal.do(cmd))
            if cmd[0] in cmdlist["translate"]:
                commands.translate.translate(self, e, cmd)
            if cmd[0] in cmdlist["rag"]:
                commands.rand.rag(self, e, cmd)
            if cmd[0] in cmdlist["ragPromo"]:
                commands.rand.ragPromo(self, e, cmd)
            if cmd[0] in cmdlist["bin"]:
                commands.cal.transbin(self, e, cmd)
            if cmd[0] in cmdlist["lotto"]:
                commands.rand.lotto(self, e, cmd)
                    
def main(): 
    username = config.twitch['bot']
    client_id = config.twitch['clientID']
    token = config.twitch['oauth']
    channel = "ekzkvh33"
    bot = TwitchBot(username, client_id, token, channel)  
    bot.start()  
    
if __name__ == "__main__":  
    main()    

{'_total': 1, 'users': [{'display_name': 'ekzkvh33', '_id': '427249143', 'name': 'ekzkvh33', 'type': 'user', 'bio': None, 'created_at': '2019-04-01T05:38:17.613591Z', 'updated_at': '2020-08-18T07:36:41.916031Z', 'logo': 'https://static-cdn.jtvnw.net/user-default-pictures-uv/de130ab0-def7-11e9-b668-784f43822e80-profile_image-300x300.png'}]}
427249143
서버 irc.chat.twitch.tv, 포트 6667에 연결중...
#ekzkvh33에 연결되었습니다.


In [ ]:
import sys, re
import irc.bot  
import requests  
import config
import time
import requests
import asyncio
import threading
from bs4 import BeautifulSoup

def get_melon(rank, site):
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
    req = requests.get(site, headers = header)
    html = req.text
    parse = BeautifulSoup(html, 'html.parser')

    titles = parse.find_all("div", {"class": "ellipsis rank01"})
    songs = parse.find_all("div", {"class": "ellipsis rank02"})

    title = []
    song = []

    for t in titles:
        title.append(t.find('a').text)

    for s in songs:
        song.append(s.find('span', {"class": "checkEllipsis"}).text)

    for i in range(rank):
        message = '!sr %s - %s'%(title[i], song[i])
        yield message

def nmsearch(search):
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
    url = "https://music.naver.com/search/search.nhn?query="
    query = search
    site = url + query + "&target=track"
    rank = 50
    req = requests.get(site, headers = header)
    html = req.text
    parse = BeautifulSoup(html, 'html.parser')

    titles = parse.find_all("a",{"class":"_title"})

    title = []
    for t in titles:
        title.append(t.find('span').text)

    for i in range(rank):
        message = '!sr %s '%(title[i])
        yield message
        
class TwitchBot(irc.bot.SingleServerIRCBot):  
    def __init__(self, username, client_id, token, channel):  
        self.client_id = client_id  
        self.token = token  
        self.channel = '#' + channel  
        self.hogumastack = 0  
  
        #채널 ID를 얻기 위해 v5 API 호출  
        url = 'https://api.twitch.tv/kraken/users?login=' + channel  
        headers = {'Client-ID': client_id, 'Accept': 'application/vnd.twitchtv.v5+json'}  
        r = requests.get(url, headers=headers).json()  
        self.channel_id = r['users'][0]['_id']  
  
        # IRC bot 연결 생성  
        server = 'irc.chat.twitch.tv'  
        port = 6667  
        print('서버 ' + server + ', 포트 ' + str(port) + '에 연결 중...')  
        irc.bot.SingleServerIRCBot.__init__(self, [(server, port, 'oauth:' + token)], username, username)  
  
    def on_welcome(self, c, e):  
        print(self.channel + '에 연결되었습니다.')  
  
        #봇을 사용하기 전에 채널 권한 부여가 필요  
        c.cap('REQ', ':twitch.tv/membership')  
        c.cap('REQ', ':twitch.tv/tags')  
        c.cap('REQ', ':twitch.tv/commands')  
        c.join(self.channel)  
  
    def on_pubmsg(self, c, e):  
        # If a chat message starts with an exclamation point, try to run it as a command
        if e.arguments[0][:1] == '!':
            cmd = e.arguments[0][1:]
            #print('Received command: ' + cmd)
            print(cmd)
            self.do_command(e, cmd)
        return
    
    def do_command(self, e, cmd):  
        c = self.connection  
        
        if cmd == "큐직명령어":
            message = "큐직봇: !탑백 !일간 !발라드 !댄스 !힙합 !트로트 !알앤비 !인디 !록 !포크 !팝백"
            c.privmsg(self.channel, message)
        elif cmd == "t":
            for i in range(100):
                time.sleep(1)
                TwitchBot.on_pubmsg(self, c, e)
                print(i)
        elif "앨범" in cmd:
            msg = cmd.split(" ")
            msg.pop(0)
            msg = str(msg[0])
            RANK = 20
            if not msg: 
                msg_choice = "검색할 앨범이 없는데요..(쭈글)"
            else:
                message = '네이버 뮤직에서 ' + msg + '님의 곡 검색중...'
                c.privmsg(self.channel, message)
                message = list(nmsearch(msg))
                for i in range(RANK):
                    c.privmsg(self.channel, message[i] + msg)
                    time.sleep(5)
                time.sleep(5)
                message = '업로드 완료!'
                c.privmsg(self.channel, message) 
        elif cmd == "탑백":
            message = '멜론에서 노래를 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/index.htm'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message) 
            
        elif cmd == "일간":
            message = '멜론에서 노래를 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message) 

        elif cmd == "발라드":
            message = '멜론에서 [일간 발라드] 를 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm?classCd=GN0100'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message)             

        elif cmd == "댄스":
            message = '멜론에서 [일간 댄스] 를 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm?classCd=GN0200'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message)  

        elif cmd == "랩" or cmd == "힙합":
            message = '멜론에서 [일간 랩/힙합] 를 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm?classCd=GN0300'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message)              
            
        elif cmd == "알앤비":
            message = '멜론에서 [일간 R&B] 를 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm?classCd=GN0400'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message)

        elif cmd == "인디":
            message = '멜론에서 [일간 인디음악] 을 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm?classCd=GN0500'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message)    

        elif cmd == "록" or cmd == "메탈":
            message = '멜론에서 [일간 록/메탈] 을 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm?classCd=GN0600'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message)                

        elif cmd == "트로트":
            message = '멜론에서 [일간 트로트] 을 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm?classCd=GN0700'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message)     

        elif cmd == "포크" or cmd == "블루스":
            message = '멜론에서 [일간 포크/블루스] 을 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/day/index.htm?classCd=GN0700'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message)     
                        
            
        elif cmd == "드라마":
            message = '멜론에서 [드라마 ost]를 받아오는 중...'
            c.privmsg(self.channel, message) 
            RANK = 50
            site = 'https://www.melon.com/chart/month/index.htm?classCd=GN1500'
            message = list(get_melon(RANK, site))
            for i in range(RANK):
                c.privmsg(self.channel, message[i])
                time.sleep(10)
            time.sleep(15)
            message = '업로드 완료!'
            c.privmsg(self.channel, message) 
            
        elif cmd == "생존":
            message = "큐직봇 생존신고 완료!"
            c.privmsg(self.channel, message)
            
    
    
def main():  
    username = "큐직" # 별 의미 없습니다. 봇 계정의 이름은 twitch 사이트에서 조정해주면 됩니다.  
    client_id = config.kuusic['clientID']# Client ID  
    token = config.kuusic['oauth'] # oauth: 는 빼고 뒷부분만 적어주시면 됩니다.  
    channel = "kuuwang" # 봇이 접속할 채널입니다. 테스트할 땐 본인의 트위치 계정을 적어주세요.
  
    bot = TwitchBot(username, client_id, token, channel)  
#    try:
    bot.start()
#    except:
#        pass
    
if __name__ == "__main__":  
    main()

서버 irc.chat.twitch.tv, 포트 6667에 연결 중...
#kuuwang에 연결되었습니다.
sr 윤도현 런데빌런 가사
sr 자우림 미안해 널 미워해
앨범 리싸
sr 리싸 유감


In [ ]:
import requests
import config
client_id = config.twitch['clientID']
url = 'https://api.twitch.tv/kraken/streams/94952468' 
headers = {'Client-ID': client_id, 'Accept': 'application/vnd.twitchtv.v5+json'}  
r = requests.get(url, headers=headers).json()  
print(r['stream']) 

In [1]:
import config
import irc.bot
import requests
import sys, re
import commands
import cmdlist
import threading

cmdlist = cmdlist.cmds
from googletrans import Translator
translator = Translator()

att = "On"
class TwitchBot(irc.bot.SingleServerIRCBot):
    def __init__(self, username, client_id, token, channel):
        self.client_id = client_id  
        self.token = token  
        self.channel = '#' + channel  

        url = 'https://api.twitch.tv/kraken/users?login=' + channel  
        headers = {'Client-ID': client_id, 'Accept': 'application/vnd.twitchtv.v5+json'}  
        r = requests.get(url, headers=headers).json()  
        self.channel_id = r['users'][0]['_id']  
        
        server = 'irc.chat.twitch.tv'
        port = 6667
        print('서버 ' + server + ', 포트 ' + str(port) + '에 연결중...')
        irc.bot.SingleServerIRCBot.__init__(self, [(server, port, 'oauth:' + token)], username, username)  
  
    def on_welcome(self, c, e):  
        print(self.channel + '에 연결되었습니다.')  
  
        c.cap('REQ', ':twitch.tv/membership')  
        c.cap('REQ', ':twitch.tv/tags')  
        c.cap('REQ', ':twitch.tv/commands')  
        c.join(self.channel)  

    def on_pubmsg(self, c, e):  
        # If a chat message starts with an exclamation point, try to run it as a command

        if att != "Off":
            sender = e.tags[4]['value']

            msg = ''.join(e.arguments)
            emoji_detector = msg.split(' ')
            emoji_file = open("emoji.txt", "r")
            emoji_list = emoji_file.readlines()
            emoji_list[:] = [emoji.rstrip('\n') for emoji in emoji_list]

            rm_emt = [word for word in emoji_detector if word not in emoji_list]
            msg = ' '.join(rm_emt)

            if sender != "Nightbot":
                if emoji_detector[0][:1] != '!':
                    if emoji_detector[0] not in emoji_list:
                        tr_detect = translator.detect(msg)
                        if str(tr_detect.lang) != 'ko':
                            if(msg.isdigit()) == False:
                                print(msg)
                                tr = translator.translate(msg, dest='en')

                                tr_results = commands.kasintr.auto_translate(tr.text)
                                print(tr_results)
                                c.privmsg(self.channel, sender + '(' + tr_detect.lang + ')' + ' -> ' + tr_results)
        if e.arguments[0][:1] == '!':
            cmd = e.arguments[0][1:]
            self.do_command(e, cmd)
        return

    def do_command(self, e, cmd):
        c = self.connection
        global att
        cmd = cmd.split(' ')
        if cmd[0] == "등록":
            with open("emoji.txt", "a+") as f:
                if cmd[1] not in f.read():
                    f.write(str(cmd).replace("'등록'","").replace(', ','\n').replace("'","").replace("[","").replace("]","").replace(',','') + "\n")
                    c.privmsg(self.channel, str(cmd).replace("'등록'","").replace("'","").replace("[","").replace("]","").replace(',','') + ' -> ' + ' 번역리스트에서 제외되었습니다.')
                    raise ValueError
        if cmd[0] in cmdlist["translate"]:
            commands.kasintr.translate(self, e, cmd)
        if cmd[0] in cmdlist["choice"]:
            commands.rand.choice(self, e, cmd)
        if cmd[0] in cmdlist["dice"]:
            commands.rand.dice(self, e, cmd)    
        if cmd[0] in cmdlist["lotto"]:
            commands.rand.lotto(self, e, cmd)
        if cmd[0] == "재부팅":
            c.privmsg(self.channel, '봇을 재부팅합니다.')
            raise ValueError
        if cmd[0] == "소환":
            att = "On"
            c.privmsg(self.channel, '누가 나를 소환했는가? 번역할 단어를 말하라!!')
        if "역" in cmd[0]:
            att = "Off"
            c.privmsg(self.channel, '조만간 너의 영혼을 가지러 오겠다!!')
        if cmd[0] in cmdlist["viewers"]:
            c.privmsg(self.channel, commands.twitchAPI.viewersJR())  
        if "뭐먹지" in cmd[0]:
            commands.rand.food(self, e, cmd) 
        if "test" in cmd[0]:
            c.privmsg(self.channel, '호에에엥')
            
            
def main():
    username = config.twitch['bot']
    client_id = config.twitch['clientID']
    token = config.twitch['oauth']
    channelg = "ghostdaa"
    channelk = "kuuwang"
    #t1 = TwitchBot(username, client_id, token, channelg)
    #botb = TwitchBot(username, client_id, token, channelk)
    #bota.start()
    #botb.start()
    #t1 = threading.Thread(target=TwitchBot, args=(username, client_id, token, channelg))
    #t2 = threading.Thread(target=TwitchBot, args=(username, client_id, token, channelk))

    #t1.setDaemon(True)
    #t2.setDaemon(True)

    t1 = TwitchBot(username, client_id, token, channelg)
    t2 = TwitchBot(username, client_id, token, channelk)
    
    t1.start()
    t2.start()
    
    threads = []
    threads.append(t1)
    threads.append(t2)
    
    for t in threads:
        t.join()
    
if __name__ == "__main__":  
    main()    


서버 irc.chat.twitch.tv, 포트 6667에 연결중...
서버 irc.chat.twitch.tv, 포트 6667에 연결중...
#ghostdaa에 연결되었습니다.


KeyboardInterrupt: 